In [ ]:
import os
import pandas as pd

# Пустой список для хранения списков лемм
all_lemmas = []

# Перебираем папки
for folder in ['1', '2', '3', '4']:
    # Путь к папке
    folder_path = os.path.join('assets/annotated_corpus/test/', folder)
    # Перебираем файлы в папке
    for file in os.listdir(folder_path):
        # Если это tsv файл
        if file.endswith('.tsv') and file.startswith('annotation'):
            # Путь к файлу
            file_path = os.path.join(folder_path, file)
            # Читаем файл
            df = pd.read_csv(file_path, sep='\t', header=None)
            # Группируем леммы по предложениям (предполагает, что предложение отделено пустой строкой)
            lemma_list = df[2].tolist()
            sentence_lemmas = []
            for lemma in lemma_list:
                if str(lemma) != 'nan':
                    sentence_lemmas.append(lemma)
                else:
                    all_lemmas.append(sentence_lemmas)
                    sentence_lemmas = []

            if len(sentence_lemmas) > 0:
                all_lemmas.append(sentence_lemmas)

In [ ]:
all_lemmas

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

data_no_stopwords = [[word for word in sentence if word not in stop_words] for sentence in all_lemmas]


In [ ]:
data_no_stopwords

### Считаем триграммы

In [ ]:
from collections import Counter

trigram_counter = Counter()

for sentence in data_no_stopwords:
    for i in range(len(sentence) - 2):
       
        # Создаем триграмму и преобразуем в строку с разделение через подчеркивание
        trigram = "_".join(sentence[i:i + 3])
        # Увеличиваем счетчик для этой триграммы
        trigram_counter[trigram] += 1


In [ ]:
sorted(trigram_counter.items(), key=lambda x:x[1], reverse=True)

### С помощью nltk

In [ ]:
from nltk.util import ngrams
from collections import Counter

# Словарь для подсчета триграмм
trigram_counter_nltk = Counter()

for sentence in data_no_stopwords:
    # Генерируем триграммы и преобразуем их в строки, сразу подсчитываем их
    trigram_counter_nltk.update("_".join(ngram) for ngram in ngrams(sentence, 3))


In [ ]:
sorted(trigram_counter_nltk.items(), key=lambda x:x[1], reverse=True)

In [ ]:
import matplotlib.pyplot as plt

# Находим топ 30 самых популярных триграмм
top_30_trigrams = trigram_counter.most_common(30)

# Распаковываем данные
trigrams, counts = zip(*top_30_trigrams)

# Создаем построение
plt.figure(figsize=(10, 8), dpi=180)
plt.barh(trigrams, counts, color='skyblue')
plt.xlabel('Частота')
plt.ylabel('Триграмма')
plt.title('Топ-30 самых популярных триграмм')
plt.gca().invert_yaxis()  # перевернуть ось Y, чтобы самая частая триграмма была наверху
plt.show()

### t-score

In [ ]:
import math
def f_absolute(text, word):
    return text.count(word)

def f_nc(word, colocate1, colocate2):  
    if word+'_'+colocate1+'_'+colocate2 in dict(sorted(trigram_counter.items(), key=lambda x:x[1], reverse=True)):
        return dict(sorted(trigram_counter.items(), key=lambda x:x[1], reverse=True))[word+'_'+colocate1+'_'+colocate2]
    else: return 0
    
def t_score(word,colocate1,colocate2,data_no_stopwords,n=3):
    all_words = [item for sublist in data_no_stopwords for item in sublist]
    N = len(all_words)
    if f_nc(word, colocate1, colocate2)==0:
        return 0
    else:
        return (f_nc(word, colocate1, colocate2) - (f_absolute(all_words,word)*f_absolute(all_words,colocate1)*f_absolute(all_words,colocate2))/N**(n-1))/math.sqrt(f_nc(word, colocate1, colocate2))
     

In [ ]:
t_score_dic = {}
for trigramm in sorted(trigram_counter.items(), key=lambda x:x[1], reverse=True)[:100]:
    ts = t_score(trigramm[0].split("_")[0], trigramm[0].split("_")[1], trigramm[0].split("_")[2] ,data_no_stopwords)
    t_score_dic[trigramm[0]] = ts


In [ ]:
sorted(t_score_dic.items(), key=lambda x:x[1], reverse=True)[:30]

### with nltk

In [ ]:
import nltk
from nltk.collocations import *
from nltk.corpus import PlaintextCorpusReader

trigram_measures = nltk.collocations.TrigramAssocMeasures()

text = [item for sublist in data_no_stopwords for item in sublist]
finder_thr = TrigramCollocationFinder.from_words(text)

# print(finder_thr.nbest(trigram_measures.student_t, 10))

for i, t_score_val in enumerate(finder_thr.score_ngrams(trigram_measures.student_t)):
    if i<30: print(t_score_val) 